<a href="https://colab.research.google.com/github/AnaP-997/product-category-classifier/blob/main/productclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🕵️‍♂️ Učitavanje i istraživanje podataka

U ovom koraku ćemo obaviti nekoliko radnji:

- Učitati CSV u Colab
- Pogledati prve i posljednje redove pomoću df.head() i df.tail()
- Provjeriti dimenzije i tipove podataka pomoću df.shape() i df.type()
- Identifikovati prazne vrijednosti pomoću df.isna().sum()
- Pogledati osnovne karakteristike i raspodjelu kategorija pomoću df.describe() i value_counts()

In [2]:
import pandas as pd

url="https://raw.githubusercontent.com/AnaP-997/product-category-classifier/main/products.csv"

# učitavanje CSV-a u Colab
df=pd.read_csv(url)

# Uvid u prvih 5 i posljednjih 5 redova
print("Uvid u prvih 5 redova: \n", df.head())
print()
print("Uvid u posljednjih 5 redova: \n",df.tail())

Uvid u prvih 5 redova: 
    product ID                                      Product Title  Merchant ID  \
0           1                    apple iphone 8 plus 64gb silver            1   
1           2                apple iphone 8 plus 64 gb spacegrau            2   
2           3  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...            3   
3           4                apple iphone 8 plus 64gb space grey            4   
4           5  apple iphone 8 plus gold 5.5 64gb 4g unlocked ...            5   

   Category Label _Product Code  Number_of_Views  Merchant Rating  \
0   Mobile Phones    QA-2276-XC            860.0              2.5   
1   Mobile Phones    KA-2501-QO           3772.0              4.8   
2   Mobile Phones    FP-8086-IE           3092.0              3.9   
3   Mobile Phones    YI-0086-US            466.0              3.4   
4   Mobile Phones    NZ-3586-WP           4426.0              1.6   

   Listing Date    
0       5/10/2024  
1      12/31/2024  
2      11/10/

In [3]:
# provjeravanje dimenzija i tipova podataka

print("Broj redova: ", df.shape[0])
print("Broj kolona:",df.shape[1])
print()

print("Tipovi kolona:\n")
print()
print(df.dtypes)

Broj redova:  35311
Broj kolona: 8

Tipovi kolona:


product ID           int64
Product Title       object
Merchant ID          int64
 Category Label     object
_Product Code       object
Number_of_Views    float64
Merchant Rating    float64
 Listing Date       object
dtype: object


In [4]:
# identifikacija praznih vrijednosti

print(df.isna().sum())

product ID           0
Product Title      172
Merchant ID          0
 Category Label     44
_Product Code       95
Number_of_Views     14
Merchant Rating    170
 Listing Date       59
dtype: int64


In [5]:
# osnovne karaktereistike i raspodjela kategorija

print(df.describe())
print()
print(df[' Category Label'].value_counts())

         product ID   Merchant ID  Number_of_Views  Merchant Rating
count  35311.000000  35311.000000     35297.000000     35141.000000
mean   26150.800176    120.501883      2501.469587         2.998261
std    13498.191220    117.045557      1438.217697         1.152664
min        1.000000      1.000000         0.000000         1.000000
25%    14958.500000     17.000000      1256.000000         2.000000
50%    27614.000000     75.000000      2509.000000         3.000000
75%    37508.500000    253.000000      3735.000000         4.000000
max    47358.000000    371.000000      5000.000000         5.000000

 Category Label
Fridge Freezers     5495
Washing Machines    4036
Mobile Phones       4020
CPUs                3771
TVs                 3564
Fridges             3457
Dishwashers         3418
Digital Cameras     2696
Microwaves          2338
Freezers            2210
fridge               123
CPU                   84
Mobile Phone          55
Name: count, dtype: int64


##🧹 Priprema i čišćenje podataka

U ovom koraku obavljamo sljedeće:

- Standardizujemo nazive kolona, jer smo primijetili neke nepravilnosti (razmake i sl.)
- Uklanjamo nepotrebne kolone koje se neće koristiti za model
- Brišemo redove s nedostajućim vrijednostima
- Standardizujemo tekst u koloni "Product Title"
- Provjeravamo i uklanjamo duplikate proizvoda ako ih ima
- Pripremamo ciljnu kolonu " Category Label"

In [6]:
# standardizacija naziva kolona

df.columns=df.columns.str.strip().str.replace(' ','_').str.lower()
print("Kolone nakon standardizacije:\n",df.columns)

# uklanjamo nepotrebne kolone (zasada su nam samo potrebne 'product_title' i 'category_label' )

df=df[['product_title','category_label']]

# brišemo redove s nedostajućim vrijednostima

df=df.dropna()

# standardizujemo tekst u koloni 'product_title'

df['product_title']=df['product_title'].str.strip().str.lower()

# provjeravamo i uklanjamo duplikate proizvoda ako ih ima

print("Duplicirani  redovi:\n",df.duplicated().sum())

Kolone nakon standardizacije:
 Index(['product_id', 'product_title', 'merchant_id', 'category_label',
       '_product_code', 'number_of_views', 'merchant_rating', 'listing_date'],
      dtype='object')
Duplicirani  redovi:
 4210


In [7]:
# uklanjanje duplih redova

df=df.drop_duplicates()
print("Broj duplikata nakon ciscenja: ",df.duplicated().sum())

# standardizacija ciljne kolone 'category_label'

df['category_label']=df['category_label'].str.strip().str.lower()

Broj duplikata nakon ciscenja:  0


## 👩‍🔬 Feature engineering na naslovima proizvoda

U ovom koraku trebamo napraviti sljedeće:

- Izračunati dužinu naslova i broj riječi u product_title
- Provjeriti prisustvo brojeva ili specijanih znakova
- Detektovati ključne pojmove ili brendove
- Kreirati jednostavne dodatne kolone koji mogu pomoći modelu da bolje razlikuje kategorije


In [11]:
# dužina naslova ili broj riječi u koloni product_title

df['len_title']=df['product_title'].str.len()
df['num_title']=df['product_title'].str.split().str.len()

# provjeravanje prisustva brojeva ili specijalnih znakova
# dodavanje novih kolona

df['has_numbers']=df['product_title'].str.contains(r'\d')
df['special_chars']=df['product_title'].str.contains(r'[^a-z0-9 ]')
print("Checking if the values have columns'; 'has_numbers' column",df['has_numbers'].head(10))
print("Checking if the values have special characters:",df['special_chars'].head(10))

# primjećujemo da se brendovi nalaze uglavnom na prvom mjestu u svakom redu
df['brand']=df['product_title'].str.split().str[0]
print(df['brand'].value_counts().head(60))

Checking if the values have columns'; 'has_numbers' column 0     True
1     True
2     True
3     True
4     True
6     True
10    True
11    True
12    True
13    True
Name: has_numbers, dtype: bool
Checking if the values have special characters: 0     False
1     False
2      True
3     False
4      True
6     False
10     True
11    False
12    False
13     True
Name: special_chars, dtype: bool
brand
bosch           2762
samsung         1969
intel           1872
siemens         1583
smeg            1273
hotpoint        1131
sony            1112
liebherr        1074
lg               979
neff             744
aeg              719
panasonic        671
beko             668
gorenje          590
canon            553
amd              547
indesit          533
zanussi          506
hoover           480
miele            464
nokia            358
hewlett          320
whirlpool        313
russell          308
nikon            304
apple            285
sharp            257
candy            253
hisen

In [13]:
brands=['bosch','samsung','intel','siemens','smeg','hotpoint','sony','liebherr','lg','panasonic','beko','gorenje','canon','zanussi','hoover','nokia','whirlpool',
        'nikon','apple','sharp','candy','hisense','bloomberg','miele','indessit','fujifilm','huawei','doro','philips','swan','amica','cda','kodak',
        'xiaomi','motorola','olympus','sim','blackberry','pentax','hk','alcatel','toshiba','iphone','daewoo','iceking']

# enkodiranje brendova u numeričke vrijednosti (pretvaranje brendova u brojeve), kako bi model mogao koristiti brand kao dodatnu numeričku karakteristiku

from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
df['brand_encoded']=le.fit_transform(df['brand'])



## 👩‍💻 Treniranje i evaluacija jednostavnog ML modela

